In [1]:
import pandas as pd
from scipy.stats import norm
from datetime import date
import math
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
import yfinance as yf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.optimize import minimize_scalar  

In [2]:
rahbek_data = pd.read_csv("../Daten/Option_prices_2018_1.csv", sep=';')
rahbek_data

,Quote date,Expiration date,C,K,S,r,T,Days to Mature
0,01.03.2016,18.03.2016,328.30,1650,1978.35,0.0033,0.046600,17
1,01.03.2016,18.03.2016,130.90,1850,1978.35,0.0033,0.046600,17
2,01.03.2016,18.03.2016,112.20,1870,1978.35,0.0033,0.046600,17
3,01.03.2016,18.03.2016,85.50,1900,1978.35,0.0033,0.046600,17
4,01.03.2016,18.03.2016,68.40,1920,1978.35,0.0033,0.046600,17
...,...,...,...,...,...,...,...,...
26432,01.03.2018,20.12.2019,171.55,2850,2677.67,0.0160,1.805479,659
26433,01.03.2018,20.12.2019,148.35,2900,2677.67,0.0160,1.805479,659
26434,01.03.2018,20.12.2019,127.00,2950,2677.67,0.0160,1.805479,659
26435,01.03.2018,18.12.2020,332.60,2700,2677.67,0.0160,2.802740,1023


In [ ]:
options_T1 = rahbek_data[rahbek_data["Days to Mature"] == 1]
options_T10 = rahbek_data[rahbek_data["Days to Mature"] == 10]
options_T30 = rahbek_data[rahbek_data["Days to Mature"] == 30]
options_T45 = rahbek_data[rahbek_data["Days to Mature"] == 45]
options_T60 = rahbek_data[rahbek_data["Days to Mature"] == 60]
options_T100 = rahbek_data[rahbek_data["Days to Mature"] == 100]

In [ ]:
def sim_option_price_sdf(series, S_t, K , horizon, par, draws, r):
    
    
    
    S_T = np.zeros(draws)
    temp = np.zeros(draws)
    
    for i in range(draws):
        ret = np.zeros(horizon)
        theta = np.zeros(horizon)
        sdf = np.zeros(horizon)
        s = np.var(series)
        
        for j in range(horizon):
            

In [ ]:
def sim_option_price_sdf(series, S_t, K, horizon, par, draws, r, model):
    """from Rasmus Petersen Copenhagen"""
    np.random.seed(123)
    omega = par[0]/100**2
    alpha = par[1]
    beta = par[2]
    mu = par[3]
        
    S_T = np.zeros(draws)
    temp = np.zeros(draws)  
    
    for i in range(draws):
        ret = np.zeros(horizon)
        theta = np.zeros(horizon)
        sdf = np.zeros(horizon)    
        s = np.var(series) #sigma2[-1]
        s =  omega + alpha*(series[-1]-mu)**2 + beta*s
            
        for j in range(horizon):
            theta[j] = (r-mu)/s - 1/2
            ret[j] = (s)**0.5 * np.random.normal(0,1,1)[0]
            sdf[j] = np.exp(theta[j]*ret[j])/np.exp( ((1+theta[j])**2 * s)/2)
            s = omega + alpha*(ret[j]-mu)**2 + beta*s    
            
        rsum = np.sum(ret)
        S_T[i] = S_t * np.exp(rsum)
        prodsdf = np.product(sdf)
        temp[i] = prodsdf * (S_T[i]-K)*(S_T[i]>K)
        
    return np.mean(temp), np.var(temp)**0.5

In [ ]:


u = 0
for element in frame_list:
    option_sub = element
    GARCH_opt_p = np.zeros(len(option_sub))
    for i in range(len(option_sub)):
        if i % 20 == 0:
            print(i)
        GARCH_opt_p[i] = sim_option_price_sdf(series = data["log_ret"], S_t = option_sub.iloc[i,4], K = option_sub.iloc[i,3], horizon = option_sub.iloc[i,7], par = np.array([0.0198,0.1003,0.8816,0.0001737859]), draws = 100, r = option_sub.iloc[i,5]/251, model = "GARCH" )[0]
    print(GARCH_opt_p)
    if u==0:
        options_T1["GARCH price"] = GARCH_opt_p
    if u==1:
        options_T10["GARCH price"] = GARCH_opt_p
    if u==2:
        options_T30["GARCH price"] = GARCH_opt_p
    if u==3:
        options_T45["GARCH price"] = GARCH_opt_p
    if u==4:
        options_T60["GARCH price"] = GARCH_opt_p
    if u==5:
        options_T100["GARCH price"] = GARCH_opt_p
    u = u+1

